<a href="https://colab.research.google.com/github/mallibus/DSS-NLP-challenge/blob/master/DSS_NLP_Challenge_Recurrent_Neural_Network_in_TensorFlow_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Installing the dependencies and setting up a GPU environment

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 45kB/s 
     |████████████████████████████████| 3.0MB 37.8MB/s 
     |████████████████████████████████| 419kB 40.7MB/s 


## Step 2: Importing the libraries

In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb

In [196]:
tf.__version__

'2.0.0-alpha0'

In [239]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
dataset_path = "/content/drive/My Drive/AIML/Projects/DSS-NLP-challenge/dataset/wiki/"
dataset_file = "dss_wiki_02000_20K1.json"

In [0]:

import json

import urllib.request

url_wiki = 'https://drive.google.com/file/d/1HZxMVthzG-tboMXO0qsPkeH2Cuq1YCbi/view?usp=sharing' # 2000

url_wiki = 'https://drive.google.com/uc?export=download&id=1iteaiSPd1OLJdKZitcv76LzrD_mAC8S8'# 5

url_wiki = 'https://drive.google.com/uc?export=download&id=15hhpN2EszdRx7-PN43yGFsoFPuZzwFO1'   # 200

f = urllib.request.urlopen(url_wiki)



In [245]:
with open(dataset_path+dataset_file, 'r') as f:
  dss = json.load(f)

print(f"Dataset of {len(dss['X'])} texts of {len(set(dss['y']))} classes - vocabulary of {len(dss['word_index'])} words")
print(dss['categories'])

Dataset of 8977 texts of 6 classes - vocabulary of 212852 words
['Economy', 'Regulation', 'Environment', 'Health', 'Industry', 'Cultures']


In [246]:
dss.keys()

dict_keys(['max_i', 'max_words', 'word_index', 'categories', 'X', 'y'])

## Step 3: Data Preprocessing

### Setting up the dataset parameters

In [0]:
number_of_words = len(dss['word_index'])
max_len = 500

In [0]:
import numpy as np

In [0]:
X_arr = np.array(dss['X'])
y_arr = np.array(dss['y'])
ylabels = dss['categories']


In [299]:
split = 0.2

dataset_size = len(y_arr)
test_size    = int(dataset_size * split)
train_size   = dataset_size - test_size
test_indexes  = np.random.choice(range(dataset_size),test_size,replace=False)
train_indexes =  np.array(list(set(range(dataset_size)) - set(test_indexes)))
np.random.shuffle(train_indexes)
dataset_size,len(train_indexes),len(test_indexes)            

(8977, 7182, 1795)

In [0]:
(X_train, y_train), (X_test, y_test) = (X_arr[train_indexes],y_arr[train_indexes]),(X_arr[test_indexes],y_arr[test_indexes])

## Explore some reviews


In [0]:
id_to_word = {dss['word_index'][key]:key for key in dss['word_index'].keys()}

In [302]:
import textwrap
size = 3
sample = np.random.choice(range(len(X_train)),size,replace=False)
for i in sample:
    print(f'Text #{i}:{ylabels[y_train[i]]}')
    s = ' '.join(id_to_word[id] for id in X_train[i])
    print('\n'.join(textwrap.wrap(s, width=120, replace_whitespace=False)))
    print()



Text #4192:Economy
jung xxxunk xxxunk chinese xxxunk pinyin xxxunk xxxunk xxxunk was a xxxunk collector dealer and scholar of traditional
chinese art xxxunk has been publicly recognized by the world congress of chinese collectors as the leading american
collector of chinese painting of his generation over his almost 50 year professional career based in the san francisco
bay area he built and helped to build several important collections of chinese painting and was friends with significant
figures in the chinese art world he authored a number of scholarly works on chinese painting and calligraphy and
collections he assembled have been the subjects of research and exhibition projects by major museums art academies and
publishing houses b xxxunk biography xxxunk was born in xxxunk china in 1929 to a prominent family that became aligned
with china's nationalist party the xxxunk his father was named xxxunk xxxunk in his early years due to the chinese civil
war he migrated to western china t

### Padding all sequences to be the same length 

In [0]:
# Changing variable name to preserve orginal dataset
X_train_p = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)

In [0]:
# Changing variable name to preserve orginal dataset
X_test_p = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

## Step 4: Building a Recurrent Neural Network

### Defining the model

In [0]:
model = tf.keras.Sequential()

### Adding the embedding layer

In [0]:
model.add(tf.keras.layers.Embedding(input_dim=number_of_words, output_dim=128, input_shape=(X_train_p.shape[1],)))

In [307]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 128)          27245056  
Total params: 27,245,056
Trainable params: 27,245,056
Non-trainable params: 0
_________________________________________________________________


### Adding the LSTM layer

- units: 128
- activation: tanh

In [308]:
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

W0720 17:41:13.333664 139751820433280 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f19a44b6860>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


### Adding the output layer

- units: 1
- activation: sigmoid

In [0]:
model.add(tf.keras.layers.Dense(units=len(ylabels), activation='softmax'))

### Compiling the model

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [311]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 128)          27245056  
_________________________________________________________________
unified_lstm_9 (UnifiedLSTM) (None, 128)               131584    
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 774       
Total params: 27,377,414
Trainable params: 27,377,414
Non-trainable params: 0
_________________________________________________________________


### Training the model

In [312]:
model.fit(X_train_p, y_train, epochs=5, batch_size=128)

Epoch 1/5
7182/7182 [==============================] - 7s 984us/sample - loss: 1.7103 - sparse_categorical_accuracy: 0.2843
Epoch 2/5
7182/7182 [==============================] - 7s 941us/sample - loss: 1.2759 - sparse_categorical_accuracy: 0.5103
Epoch 3/5
7182/7182 [==============================] - 7s 946us/sample - loss: 0.8762 - sparse_categorical_accuracy: 0.7196
Epoch 4/5
7182/7182 [==============================] - 7s 943us/sample - loss: 0.4669 - sparse_categorical_accuracy: 0.8592
Epoch 5/5
7182/7182 [==============================] - 7s 941us/sample - loss: 0.2717 - sparse_categorical_accuracy: 0.9198


### Evaluating the model

In [313]:
test_loss, test_acurracy = model.evaluate(X_test_p, y_test)

1795/1795 [==============================] - 2s 860us/sample - loss: 0.8615 - sparse_categorical_accuracy: 0.7337


In [314]:
print("Test accuracy: {}".format(test_acurracy))

Test accuracy: 0.7337047457695007


In [0]:
#model = tf.keras.Sequential()
#model.add(tf.keras.layers.Embedding(input_dim=number_of_words, output_dim=128, input_shape=(X_train_p.shape[1],)))
#model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))
#model.add(tf.keras.layers.GlobalAveragePooling1D())
#model.add(tf.keras.layers.Dense(units=128, activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(0.2))
#model.add(tf.keras.layers.Dense(units=128, activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(0.2))
#model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
#model.fit(X_train_p, y_train, epochs=3, batch_size=128)
#test_loss, test_acurracy = model.evaluate(X_test_p, y_test)
#print("Test accuracy: {}".format(test_acurracy))

In [315]:

size = 3
sample = np.random.choice(range(len(X_test)),size)
predictions = model.predict_classes(X_test_p[sample])
for idx,i in enumerate(sample):
    print(f'Review #{i} - P:{ylabels[predictions[idx]]},A:{ylabels[y_test[sample[idx]]]}')
    s = ' '.join(id_to_word[id] for id in X_test[i])
    print('\n'.join(textwrap.wrap(s, width=120, replace_whitespace=False)))
    print()


Review #333 - P:Regulation,A:Regulation
regulatory law refers to law promulgated by an executive branch agency under a delegation from a legislature discussed
at regulation law and primary and secondary legislation regulatory law contrasts to statutory law promulgated by the
legislative branch and common law or case law promulgated by the judicial branch the law that governs conduct of
administrative agencies both xxxunk of regulations and xxxunk of applications or disputes and judicial review of agency
decisions is called administrative law administrative law is promulgated by the legislature and refined by judicial
common law for governing agencies the administrative agencies are to create procedures applications licences appeals and
one other important thing called decision making administrative procedure act is responsible for all the federal agency
policies some states have started following the policies through law or by regulation references

Review #288 - P:Economy,A:Economy
di